In [1]:
import sys
!{sys.executable} -m pip install pulp 

import pulp
# Import PuLP modeler functions
from pulp import * 

import numpy
import matplotlib as plt
import pandas as pd

# Seaborn for pretty graphs
import seaborn



## Context: Marine Spatial Planning and a Linear Programming Approach

## The Decision Variables

In [2]:
"""
Strait of Georgia, Northern Shelf, Southern Shelf and Offshore Coast are the 4 main bioregions localized to the 
BC region.
"""

bioregions = ['SG', 'NS', 'SS', 'OC']

In [3]:
prob = LpProblem("Marine_Spatial_Planning_BC",LpMinimize)

In [4]:
#lowBound = 0 to enforce nonnegativity
msp_areas = LpVariable.dicts("br",bioregions,lowBound=0)

## The Objective Function

In [5]:
prob+=lpSum([msp_areas[i] for i in bioregions]),"Minimum Area Required for Conservation Use"

## The Constraints

In [6]:
# PROJECTED NUMBERS

#Strait of Georgia = 6,800 km2
#Northern Shelf = 101,000 km2
#Southern Shelf = 41, 562 km2
#Offshore Coast = 335,825 km2

bioregion_area = {'SG': 6800,
                  'NS': 101000,
                  'SS': 41562,
                  'OC': 133021}

for j in msp_areas:
    prob += msp_areas[j] <= bioregion_area[j]


def formulateConstraint(percentage, areas):
    req = percentage * lpSum ( [ areas[i] * bioregion_area[i] for i in areas] )
    return lpSum([areas[i] * msp_areas[i] for i in msp_areas]) >= req

    

#60.95% of combined land in strait of Georgia, Northern and Southern shelf should accomodate seabirds
seabird_percentage = 0.6095
seabird_areas = { 'SG': 1,
                  'NS': 1,
                  'SS': 1,
                  'OC': 0}

prob += formulateConstraint(seabird_percentage, seabird_areas),"Seabird"


#71.86% of combined land in Northern and Southern shelf should accomodate marine fish life and plantlife
shelf_fish_percentage = 0.7186
shelf_fish_areas = {'SG': 0,
                    'NS': 1,
                    'SS': 1,
                    'OC': 1}

prob += formulateConstraint(shelf_fish_percentage, shelf_fish_areas), "Fish"



#44.16% of combined land in all bio-regions but the Strait of Georgia
mammal_percentage = 0.4416
mammal_areas = {    'SG': 1,
                    'NS': 1,
                    'SS': 0,
                    'OC': 1}

prob += formulateConstraint(mammal_percentage, mammal_areas), "Mammal"


In [7]:
print(prob)

#LP also written to .lp file for convenience
prob.writeLP("BC_MSP.lp")

Marine_Spatial_Planning_BC:
MINIMIZE
1*br_NS + 1*br_OC + 1*br_SG + 1*br_SS + 0
SUBJECT TO
_C1: br_SG <= 6800

_C2: br_NS <= 101000

_C3: br_SS <= 41562

_C4: br_OC <= 133021

Seabird: br_NS + br_SG + br_SS >= 91036.139

Fish: br_NS + br_OC + br_SS >= 198033.9438

Mammal: br_NS + br_OC + br_SG >= 106346.5536

VARIABLES
br_NS Continuous
br_OC Continuous
br_SG Continuous
br_SS Continuous



[br_NS, br_OC, br_SG, br_SS]

## Solution to the LP

In [8]:
prob.solve()
print("Status:",LpStatus[prob.status])

for a in prob.variables():
    print(a.name,"=",a.varValue)
    
print("Optimal area amount = ",value(prob.objective))

Status: Optimal
br_NS = 49474.139
br_OC = 106997.8
br_SG = 0.0
br_SS = 41562.0
Optimal area amount =  198033.939


### Graphical Analysis of the Solution

## Accounting for minute changes in our variables - Sensitivity Analysis

In [9]:
print("\nSensitivity Analysis\nConstraint\t\t\t\tShadow Price\t\t\tSlack")
for name, c in prob.constraints.items():
    print(name, ":", c, "\t\t\t", c.pi, "\t\t\t\t", c.slack)



Sensitivity Analysis
Constraint				Shadow Price			Slack
_C1 : br_SG <= 6800 			 0.0 				 6800.0
_C2 : br_NS <= 101000 			 0.0 				 51525.861
_C3 : br_SS <= 41562 			 0.0 				 -0.0
_C4 : br_OC <= 133021 			 0.0 				 26023.199999999997
Seabird : br_NS + br_SG + br_SS >= 91036.13900000001 			 0.0 				 1.4551915228366852e-11
Fish : br_NS + br_OC + br_SS >= 198033.9438 			 1.0 				 0.0038000000058673322
Mammal : br_NS + br_OC + br_SG >= 106346.5536 			 0.0 				 -50125.3864
